In [1]:
# Train and save lstm model
import pandas as pd
import numpy as np
import jieba
import multiprocessing
from os import path

In [2]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import gensim
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence

from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout, Activation
from keras.models import model_from_yaml
np.random.seed(1337)
import sys
sys.setrecursionlimit(1000000)
import yaml

c:\users\82557\documents\github\weibo_analysis\venv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
c:\users\82557\documents\github\weibo_analysis\venv\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
cpu_count = multiprocessing.cpu_count() # 4
vocab_dim = 100
n_iteration = 1 # as more as possible
n_exposures = 10
window_size = 7
n_epoch = 5
input_length = 100
maxlen = 100 # 文本保留最大长度
batch_size = 100
n_cpu = multiprocessing.cpu_count()

In [4]:
import logging
logger = logging.getLogger('lstm')
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
word2vec_file = path.join('demo','model','wiki.vector')
wiki_file = path.join('demo','data','wiki.zh.simple.seg.txt')
print("training word2vec model...")
#word2vec_model = Word2Vec(LineSentence(wiki_file), size=100, min_count=5, window=window_size, workers=n_cpu)
# word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file, binary=False)
print("training is done.")

training word2vec model...
training is done.


In [ ]:
# print("saving word2vec model to {}...".format(word2vec_file))
# word2vec_model.wv.save_word2vec_format(word2vec_file, binary=False)
# word2vec_model.save(path.join('demo','model','wiki.model'))

In [5]:
from gensim.corpora.dictionary import Dictionary
def create_dictionary(word2vec_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(word2vec_model.wv.vocab.keys(), allow_update=True)
    word2index = {v:k+1 for k,v in gensim_dict.items()}
    word2vec = {word:word2vec_model[word] for word in word2index.keys()}
    return word2index, word2vec

def parse_dataset(combined, w2index): # 
    ''' Words become integers
    '''
    data=[]
    for sentence in combined:
        new_txt = []
        for word in sentence:
            try:
                new_txt.append(w2indx[word])
            except:
                new_txt.append(0) # freqxiao10->0
        data.append(new_txt)
    return data # word=>index

def word2vec_train(combined):
                   
    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=n_cpu,iter=n_iteration)
    model.build_vocab(combined) # input: list
    model.train(combined, total_examples=model.corpus_count, epochs=model.iter)
    model.save(path.join('model','w2v_model.pkl'))
    index_dict, word_vectors = create_dictionary(model)
    return   index_dict, word_vectors,combined,model


In [6]:
import pickle
# index_dict, word_vectors = create_dictionary(word2vec_model)  # index_dict: {word:index} word_vectors:{word:vector}
# output1 = open(path.join('model','index_dict.pkl'), 'wb')
# output2 = open(path.join('model','word_vectors.pkl'), 'wb')
# pickle.dump(index_dict, output1)
# pickle.dump(word_vectors, output2)
# index_dict, wordvetors = create_dictionary(word2vec_model)

In [7]:
import string, re

In [8]:
def clean_content(sentence):
    if sentence != '':
        sentence = sentence.strip()
        in_tab = b''
        out_tab = b''
        trans_tab = bytearray.maketrans(in_tab, out_tab)
        pun_digit = string.punctuation + string.digits
        pun_digit = pun_digit.encode('utf-8')
        sentence = sentence.encode('utf-8')
        sentence = sentence.translate(trans_tab, pun_digit)
        sentence = sentence.decode('utf-8')
        sentence = re.sub('[a-zA-Z0-9]', '', sentence)
        sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\'；：“”．]+|[+——！，。？?、~@#￥%……&*（）《》【】]+", "", sentence) 
    return sentence

def sent2word(sentence):
    seg_list = jieba.cut(sentence, cut_all=False)
    seg_result = ''
    for word in seg_list:
        if word != '\t':
            seg_result += word + ' '
    return seg_result.strip()

def words2vec(sentence, model):
    vecs = []
    sentence = clean_content(sentence)
    words = sent2word(sentence)
    for word in words:
        word = word.replace('\n', '')
        try:
            vecs.append(model[word])
        except KeyError:
            continue
    return np.array(vecs, dtype='float')

def build_matrix(train_set, model, avg=True):
    x_train = []
    y_train = []
    i = 1
    print("Building matdix....")
    for row in train_set:
        if i % 1000 == 0:
            print("Building matrix at row {}.".format(i))
        i += 1
        if pd.isna(row[0]):
            continue
        vecs = words2vec(row[0], model)
        if len(vecs) > 0:
            if avg:
                vecs_array = sum(np.array(vecs)/len(vecs)) # Use avg to reduce caltulate cost.
                x_train.append(vecs_array)
            y_train.append(int(row[1]))
    return x_train, y_train


def build_train_set(train_set):
    x_train = []
    y_train = []
    i = 1
    print("Building train data...")
    for row in train_set:
        if i%500 == 0:
            print("Building train data at row{}.".format(i))
        i+=1
        if pd.isna(row[0]):
            continue
        sentence = clean_content(row[0])
        x_train.append(jieba.lcut(sentence.replace('\n','')))
        y_train.append(int(row[1]))
    return x_train, y_train

In [9]:
df = pd.read_csv(path.join('2015_core', 'OriginalTrain.csv'), header=None, index_col=None)
df.head(100)

,0,1,2,3,4,5,6,7
0,1UserName,2TopicID,3DocID,4ThreadID,5Parent,6Children,7Text,8Label
1,南加大CPD,0,0,3731074750277011,-1,"1,2,4,5",#网络战中的虫群战术# 玩过“星际争霸”这款游戏的人都应该知道虫群的厉害。如今，伊拉克反政府...,0
2,狗尾巴花花,0,1,3731074750277011,0,NaN,不知道运用新媒体，那就是自己制造灾难！,-1
3,南加大CPD,0,2,3731074750277011,0,3,#悉尼劫持案# 应当让国人认识到ISIS（伊黎）对世界和平的威胁程度可能要比你想象中的高。如...,0
4,海ER哥我是克鲁德,0,3,3731074750277011,2,NaN,推特这种地方能一直允许这些动物堂而皇之的开设账号进行广播，本来就挺吊诡的。,-1
5,南加大CPD,0,4,3731074750277011,0,NaN,#法国杂志社遇袭# 恐怖分子利用社交媒体招兵买马，发动“独狼式”恐怖袭击,0
6,snowflying_chen,0,5,3731074750277011,0,NaN,技术好高超,0
7,海尔冷柜冻之道,0,0,3752859822643460,-1,"1,4,5,6,7,8",朋友们，期待很久了吧，悦冻·帅富豪闪耀上市了！风冷无霜百变空间，深度冷藏时刻新鲜，即日起至9...,0
8,全球热门搞笑汇,0,1,3752859822643460,0,"2,3",站着的冰柜，你见过么？,0
9,布鲁特多雅,0,2,3752859822643460,1,NaN,这个棒[花心]还是无霜的哈哈，我来预约了,0


In [10]:
x_content = df[6]
print(x_content)
y_train = df[7].values[1:]
print(y_train)
train_set = df[[6,7]].values[1:]
print(train_set)

0                                                   7Text
1       #网络战中的虫群战术# 玩过“星际争霸”这款游戏的人都应该知道虫群的厉害。如今，伊拉克反政府...
2                                     不知道运用新媒体，那就是自己制造灾难！
3       #悉尼劫持案# 应当让国人认识到ISIS（伊黎）对世界和平的威胁程度可能要比你想象中的高。如...
4                   推特这种地方能一直允许这些动物堂而皇之的开设账号进行广播，本来就挺吊诡的。
5                    #法国杂志社遇袭# 恐怖分子利用社交媒体招兵买马，发动“独狼式”恐怖袭击
6                                                   技术好高超
7       朋友们，期待很久了吧，悦冻·帅富豪闪耀上市了！风冷无霜百变空间，深度冷藏时刻新鲜，即日起至9...
8                                             站着的冰柜，你见过么？
9                                    这个棒[花心]还是无霜的哈哈，我来预约了
10                                          不错！好想买回家[笑哈哈]
11                                            站着的冰柜，你见过么？
12                                                  这个还不错
13                                             真的是酷毙了[嘻嘻]
14                                      [嘻嘻][太开心]这冰柜果然够气派
15                                       赶紧买回家，正好要装修房子哈哈哈
16                                   这个棒[花心]还是无霜的哈哈，我来预约了
17      【同粉互动】

In [11]:
# x_train, y_train = build_matrix(train_set, model=word2vec_model)
x_train, y_train = build_train_set(train_set)
print(x_train)

Building prefix dict from the default dictionary ...
2018-05-09 23:40:35,381: DEBUG: Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\82557\AppData\Local\Temp\jieba.cache
2018-05-09 23:40:35,386: DEBUG: Loading model from cache C:\Users\82557\AppData\Local\Temp\jieba.cache


Building train data...


Loading model cost 0.924 seconds.
2018-05-09 23:40:36,308: DEBUG: Loading model cost 0.924 seconds.
Prefix dict has been built succesfully.
2018-05-09 23:40:36,309: DEBUG: Prefix dict has been built succesfully.


Building train data at row500.
Building train data at row1000.
Building train data at row1500.
Building train data at row2000.
Building train data at row2500.
[['网络战', '中', '的', '虫', '群', '战术', '玩过', '星际争霸', '这', '款', '游戏', '的', '人', '都', '应该', '知道', '虫', '群', '的', '厉害', '如今', '伊拉克', '反', '政府', '武装', '伊拉克', '和', '黎凡特', '伊斯兰', '国', '正在', '运用', '类似', '的', '战术', '培养', '大批', '媒体', '圣战者', '运用', '社交', '媒体', '进行', '百团大战', '式', '的', '传播', '通过', '数据分析', '如何', '利用', '社交', '媒体', '进行', '舆论', '造势'], ['不', '知道', '运用', '新', '媒体', '那', '就是', '自己', '制造', '灾难'], ['悉尼', '劫持', '案', '应当', '让', '国人', '认识', '到', '伊黎', '对', '世界', '和平', '的', '威胁', '程度', '可能', '要', '比', '你', '想象', '中', '的', '高', '如今', '社交', '媒体', '已经', '成为', '了', '恐怖分子', '招兵买马', '动摇', '年轻人', '思想', '的', '阵地', '特约', '作者', '运用', '数据', '为', '我们', '展开', '利用', '社交', '媒体', '传播', '极端', '思想', '的', '路线图'], ['推特', '这种', '地方', '能', '一直', '允许', '这些', '动物', '堂而皇之', '的', '开设', '账号', '进行', '广播', '本来', '就', '挺', '吊', '诡', '的'], ['法国', '杂志社', '遇袭', '恐怖分子', '利用',

In [12]:
print("Building is done.")
index_dict, word_vectors,x_train, w2v_model = word2vec_train(x_train)
print(len(x_train[0]))
print(len(x_train))

2018-05-09 23:40:36,956: INFO: collecting all words and their counts
2018-05-09 23:40:36,957: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-09 23:40:36,977: INFO: collected 11143 word types from a corpus of 45615 raw words and 2812 sentences
2018-05-09 23:40:36,979: INFO: Loading a fresh vocabulary
2018-05-09 23:40:36,990: INFO: min_count=10 retains 660 unique words (5% of original 11143, drops 10483)
2018-05-09 23:40:36,991: INFO: min_count=10 leaves 25974 word corpus (56% of original 45615, drops 19641)
2018-05-09 23:40:36,994: INFO: deleting the raw counts dictionary of 11143 items
2018-05-09 23:40:36,996: INFO: sample=0.001 downsamples 65 most-common words
2018-05-09 23:40:36,997: INFO: downsampling leaves estimated 18375 word corpus (70.7% of prior 25974)
2018-05-09 23:40:37,000: INFO: estimated required memory for 660 words and 100 dimensions: 858000 bytes
2018-05-09 23:40:37,001: INFO: resetting layer weights
c:\users\82557\documents\github\weib

Building is done.
56
2812


c:\users\82557\documents\github\weibo_analysis\venv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [13]:
word_dim = 100 # 向量维度
input_lenght = 100
batch_size = 32
maxlen = 100 # 文本保留最大长度
drop = 0.5
n_epoch = 5
def train_lstm(n_symbols, embedding_weights, x_train, y_train, x_test, y_test):
    print("Building lstm model...")
    model = Sequential()
    model.add(Embedding(output_dim=word_dim,
                       input_dim = n_symbols,
                       mask_zero=True,
                       weights=[embedding_weights],
                       input_length=input_length))
    model.add(LSTM(output_dim=50,
                  activation='tanh',
                  inner_activation='hard_sigmoid'))
    model.add(Dropout(drop))
    model.add(Dense(3,activation='softmax'))
    model.add(Activation('softmax'))
    print("Compiling the model...")
    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    print("Strat training...")
    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch, verbose=1)
    print("Evaluating...")
    score, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
    print("Test score:{}".format(score))
    print("Test accuracy:{}".format(accuracy))
    print("saving model...")
    yaml_string = model.to_yaml()
    with open(path.join('model','lstm.yml'), 'w') as f:
        f.write(yaml.dump(yaml_string, default_flow_style=True))
    model.save_weights(path.join('model','lstm.h5'))
    print("saving done.")
    
def text2index_array(index_dict, sentences):
    # document to index number
    new_sentence = []
    for sen in sentences:
        new_sen = []
        for word in sen:
            try:
                new_sen.append(index_dict[word])
            except KeyError:
                new_sen.append(0)
        new_sentence.append(new_sen)
    return np.array(new_sentence)


In [14]:
import keras
x_train = text2index_array(index_dict, x_train)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
y_train = keras.utils.to_categorical(y_train, num_classes=3)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.20)
n_symbols = len(index_dict)+1
embedding_weights = np.zeros((n_symbols, vocab_dim)) # 初始化 索引为0的词语，词向量全为0
# index_dict: {word:index} word_vectors:{word:vector}
index = 0
for word, _ in index_dict.items():# 从索引为1的词语开始，对每个词语对应其词向量
    try:
        embedding_weights[index, :] = word_vectors[word]
    except KeyError:
        continue
    index += 1

In [16]:

print("X_train shape: {}".format(x_train.shape))
print("X_test shape:{}".format(x_test.shape))
import keras
print(x_train[1])

X_train shape: (2249, 100)
X_test shape:(563, 100)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 195 243   7 456 171 155 557   0   0 353 360   0   0   0
   0   0 388 477   0   0   0 187   0 331   0   0   0   0 396 552   0   0
   0 489   0 139  93   0 489   0   0 602]


In [17]:
train_lstm(n_symbols,embedding_weights,x_train, y_train, x_test, y_test)

Building lstm model...


c:\users\82557\documents\github\weibo_analysis\venv\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, recurrent_activation="hard_sigmoid", activation="tanh")`


Compiling the model...
Strat training...
Epoch 1/5
2249/2249 [==============================] - 6s 3ms/step - loss: 0.6325 - acc: 0.6667
Epoch 2/5
2249/2249 [==============================] - 5s 2ms/step - loss: 0.6025 - acc: 0.6770
Epoch 3/5
2249/2249 [==============================] - 5s 2ms/step - loss: 0.5566 - acc: 0.7261
Epoch 4/5
2249/2249 [==============================] - 5s 2ms/step - loss: 0.5288 - acc: 0.7658
Epoch 5/5
2249/2249 [==============================] - 5s 2ms/step - loss: 0.5119 - acc: 0.7889
Evaluating...
563/563 [==============================] - 0s 880us/step
Test score:0.5763174475403916
Test accuracy:0.7128478580850903
saving model...
saving done.


In [18]:
lstm_file = path.join('model','lstm.yml')
with open(lstm_file, 'r') as f:
    yaml_string = yaml.load(f)
lstm_model = model_from_yaml(yaml_string)
lstm_model.load_weights(path.join('model','lstm.h5'))
lstm_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])

def lstm_predict(content, lstm_model, w2v_model):
    words = jieba.lcut(clean_content(content))
    words = np.array(words).reshape(1,-1)
    data = text2index_array(index_dict,words)
    data = sequence.pad_sequences(data, maxlen=maxlen)
    data.reshape(1,-1)
    print(data.shape)
    result = lstm_model.predict_classes(data)
    print(result)
    if result[0] == 1:
        print("pos")
    elif result[0] == 0:
        print("neu")
    elif result[0] == 2:
        print("neg")
    

In [20]:
content = "你做的菜真好吃，我很喜欢很高兴"
lstm_predict(content, lstm_model,w2v_model)

(1, 100)
[1]
pos
